In [1]:
import pandas as pd
data = pd.read_csv('IsusV3.csv', delimiter = ',').reset_index(drop=True)
train = pd.read_csv('train.csv', delimiter = ',').reset_index(drop=True)
test = pd.read_csv('test.csv', delimiter = ',').reset_index(drop=True)

data1 = data.drop(['Target'], axis=1).fillna(0)
# data1 = data1.values
y = data[['Target']].values
data1.shape
# y
x_train_ = train.drop(['Target'], axis=1).fillna(0)
y_train = train[['Target']].values

x_test_ = test.drop(['Target'], axis=1).fillna(0)
y_test = test[['Target']].values
x1 = pd.concat([x_train_, x_test_])

x = data1.to_dict('records')
x_train = x_train_.to_dict('records')
# y_train = y_train_.to_dict('records')

x_test = x_test_.to_dict('records')
# y_test = y_test_.to_dict('records')

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np

vec = DictVectorizer()
x = vec.fit_transform(x).toarray()

x_train = vec.transform(x_train).toarray()
x_test = vec.transform(x_test).toarray()

In [3]:
from imblearn.over_sampling import SMOTE, ADASYN
# smote = SMOTE(ratio='minority')
# sm = SMOTE(random_state=42)
sm = ADASYN(random_state=42)
x_sm, y_sm = sm.fit_sample(x_train, y_train)
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_sm.shape)
print(x_sm.shape,y_sm.shape)
x_val = x_test
y_val = y_test

C:\Users\EBM_IT_01\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(8576, 172) (8576, 1) (2144, 172) (2144, 1) (16093, 172)
(16093, 172) (16093,)


In [4]:
from keras.models import Model
from keras.layers import Input, Dense,Embedding,GRU,Dropout,Conv1D,MaxPooling1D,Flatten,concatenate,Add,Reshape,BatchNormalization
from keras.optimizers import Adam , RMSprop
import keras.utils

def gru_model():
  input = Input(shape=(172,))
  x = Dropout(0.10)(input)
  x = Dense(30, activation='relu')(x)
  x = Dropout(0.10)(x)
  for i in range(0, 50):
      x = Dense(10, activation='relu')(x)
      x = Dropout(0.10)(x)
#   x = Dense(10, activation='relu')(x)
#   x = Dropout(0.10)(x)
#   x = Dense(10, activation='relu')(x)
#   x = Dropout(0.10)(x)
  out = Dense(1, activation='sigmoid')(x)
  

  model = Model(inputs=[input], outputs=out)
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

C:\Users\EBM_IT_01\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
K.clear_session()
weight_path_feedforward_nn='recomment.h5'
callbacks_list_feedforward_nn = [
#         TensorBoard(log_dir='/data/Graph/ff', histogram_freq=1, write_graph=True, write_grads=False),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]
print('start training GRU')
model_ = gru_model()
verbose = 1
train_params = [(20 ,28)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_.fit(x_sm, y_sm, epochs=epochs, batch_size=batch_size, verbose=verbose,
#                            callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_test, y_test))

start training GRU
train with 20 epochs and 28 batch size
Train on 16093 samples, validate on 2144 samples
Epoch 1/20
16093/16093 [==============================] - 9s 544us/step - loss: 0.6932 - acc: 0.4989 - val_loss: 0.6892 - val_acc: 0.9459
Epoch 2/20
16093/16093 [==============================] - 3s 185us/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6904 - val_acc: 0.9459
Epoch 3/20
16093/16093 [==============================] - 3s 187us/step - loss: 0.6932 - acc: 0.4989 - val_loss: 0.6876 - val_acc: 0.9459
Epoch 4/20
16093/16093 [==============================] - 3s 187us/step - loss: 0.6932 - acc: 0.4998 - val_loss: 0.6887 - val_acc: 0.9459
Epoch 5/20
16093/16093 [==============================] - 3s 184us/step - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6902 - val_acc: 0.9459
Epoch 6/20
16093/16093 [==============================] - 3s 212us/step - loss: 0.6932 - acc: 0.5028 - val_loss: 0.6861 - val_acc: 0.9459
Epoch 7/20
16093/16093 [==============================] - 4s 226u

In [9]:
model_.predict(x_test)

array([[0.49731335],
       [0.49731335],
       [0.49731335],
       ...,
       [0.49731335],
       [0.49731335],
       [0.49731335]], dtype=float32)

In [10]:
from sklearn.metrics import f1_score , precision_score , recall_score , accuracy_score , roc_curve , auc , classification_report
# y_pre = bst.predict(dt, ntree_limit=bst.best_ntree_limit)
threshold = 0.8
# treee = bst.best_ntree_limit
# treee = 19
y_pre = model_.predict(x_test)
y_pre  = y_pre > threshold
y_pre = y_pre.astype(int) 
f1_1  = f1_score(y_test, y_pre, average='binary')


# y_pre_2 = bst.predict(dtrain, ntree_limit=bst.best_ntree_limit)
y_pre_2 = model_.predict(x_train)
y_pre_2  = y_pre_2 > threshold
y_pre_2 = y_pre_2.astype(int) 
f1_2  = f1_score(y_train, y_pre_2, average='binary')
print(f1_1,f1_2)
print(classification_report(y_test, y_pre))

C:\Users\EBM_IT_01\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0 0.0
             precision    recall  f1-score   support

        0.0       0.95      1.00      0.97      2028
        1.0       0.00      0.00      0.00       116

avg / total       0.89      0.95      0.92      2144



C:\Users\EBM_IT_01\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
